In [7]:
import model
import pandas as pd
import numpy as np

In [52]:
def counterterror_model_http(request):
    """
    HTTP Cloud Function
    Args:
        request (flask.Request): request object providing parameters for model
    Returns:

    """

    #request_json = request.get_json(silent=True)
    #request_args = request.args
    request_args = request

    grid_size = {200:(2,3),300:(3,3),400:(3,4),500:(4,4),600:(4,5),\
                800:(5,5)}
    h, w = grid_size[request['starting_pop']]

    scenario = model.CounterterrorismModel(N=request['starting_pop'], \
            height=h,width=w,prob_violence=request['violence_prob'],\
            policy=request['gov_policy'].upper(),reactive_lvl=\
            request['reactive_level'].lower(),\
            discontent=request['discontent'].lower(),add_violence_aftermath=\
            request['add_violence_aftermath'],crowding_threshold=request['crowding_threshold'],\
            agent_birth_rate=request['agent_birth_rate'])
    for i in range(request['total_steps']):
        scenario.step()

    # create arrays to send to dashboard
    max_y_govtactions = []
    hMap_act_x = []
    hMap_act_y = []
    hMap_act_size = []
    hMap_act_symbol = []
    step = list(range(1,request['total_steps']+1))
    hMap_sympathy = []
    hMap_num_combats = []

    pChart_classes = []

    # ----- Create dominant sentiment and num_combatants grids --- #
    agents_df = scenario.datacollector.get_agent_vars_dataframe()
    agents_df = agents_df.reset_index()
    agents_nogov_df = agents_df[agents_df.status.isin(['anti-violence',\
        'combatant','neutral','sympathetic'])]
    agents_nogov_df['step_cat'] = agents_nogov_df.Step.astype('str')
    
    # create df of num agents of each status per step
    agent_stati_gb = agents_nogov_df.groupby(by=['step_cat','status'])
    agent_stati = agent_stati_gb['AgentID'].nunique().reset_index()
    agent_stati = agent_stati.rename(columns={'AgentID':'num_agents'})
    agent_stati['step'] = agent_stati.step_cat.astype('int')

    # create df of dominant sentiment per cell per step
    agent_sents_gb = agents_nogov_df.groupby(by=['step_cat','agent_loc','status'])
    sentiments_cnt = agent_sents_gb['AgentID'].nunique().reset_index()
    sentiments_cnt = sentiments_cnt.rename(columns={'AgentID':'num_agents'})
    max_sentiments_gb = sentiments_cnt.groupby(by=['step_cat','agent_loc'])
    max_sentiments = max_sentiments_gb['num_agents'].max().reset_index()
    #print('max sent shape',max_sentiments.shape)
    max_sentiments = max_sentiments.merge(sentiments_cnt, on=['step_cat','agent_loc','num_agents'])
    # randomly decide which to keep for now; ideally drop according to hierarchy in future iteration
    max_sents_nodups = max_sentiments.drop_duplicates(subset=['step_cat','agent_loc','num_agents'])
    max_sents_nodups['step'] = max_sents_nodups.step_cat.astype('int')
    sentiments_cnt['step'] = sentiments_cnt.step_cat.astype('int')

   # ----- Create govt action arrays ----- #
    govt_actions = scenario.datacollector.get_table_dataframe('govt_actions')
    max_num_govt_actions = 0   
    full_xs = ['A','B','C','D','E','F']
    scen_xs = full_xs[:scenario.grid.width]
    full_ys = ["ㅤ1", "ㅤ2", "ㅤ3","ㅤ4", "ㅤ5", "ㅤ6"]        
    scen_ys = full_ys[:scenario.grid.height]     

    for mod_step in range(1,request['total_steps']+1):
        print('mod step',mod_step,'type',type(mod_step))
        agent_step_df = agent_stati[agent_stati.step == mod_step]

        # update pChart classes for each step
        pChart_classes_step = [0,0,0,0]
        stati_map = {'anti-violence':0.0,'neutral':0.25,
                'sympathetic':0.50,'combatant':0.75}
        stati = ['anti-violence','neutral','sympathetic','combatant']
        for idx, stati_cnt in enumerate(stati):
            # if agents of status exist, set to number, else assign 0
            if stati_cnt in list(agent_step_df.status):
                pChart_classes_step[idx] = agent_step_df[agent_step_df.status==stati_cnt].\
                    num_agents.item()
        
        print('pChart classes to add',pChart_classes_step)
        pChart_classes.append(pChart_classes_step)

        # update dominant sentiments and num combatants per step
        max_sent_step_df = max_sents_nodups[max_sents_nodups.step == mod_step]
        hmap_sentiments = np.zeros((scenario.grid.width, scenario.grid.height))
        hmap_num_combatants = np.zeros((scenario.grid.width, scenario.grid.height))
        for hmap_loc in max_sent_step_df.agent_loc:
            hmap_sentiments[hmap_loc] = stati_map[max_sent_step_df[max_sent_step_df.agent_loc==\
                                        hmap_loc].status.item()]

            
            test_comb_df = sentiments_cnt[(sentiments_cnt.step == mod_step) & \
                        (sentiments_cnt.agent_loc == hmap_loc)]
            if 'combatant' in list(test_comb_df.status):
                hmap_num_combatants[hmap_loc] = test_comb_df[test_comb_df.status == \
                                    'combatant'].num_agents.item()
            else:
                hmap_num_combatants[hmap_loc] = 0
        
        print('hMap sympathy to append',hmap_sentiments)
        hMap_sympathy.append(hmap_sentiments.tolist())

        print('hMap num combatants to append',hmap_num_combatants)
        hMap_num_combats.append(hmap_num_combatants.tolist())


        # update gov't action per step
        hmap_x = []
        hmap_y = []
        hmap_size = []
        hmap_symbol = []
        govt_action_df = govt_actions[govt_actions.step == mod_step]
        this_step_action = govt_action_df.govt_action.item()
        print('this step action',this_step_action)
        if this_step_action in ['INDISC-CONC','INDISC-REPR']:
            max_num_govt_actions += 1
            hmap_x_long = [[x] * scenario.grid.height for x in scen_xs]
            hmap_x = [val for sublist in hmap_x_long for val in sublist]
            hmap_y = scen_ys * scenario.grid.width
            hmap_size = [80] * scenario.grid.width * scenario.grid.height
            if this_step_action == 'INDISC-CONC':
                hmap_symbol = [105] * scenario.grid.width * scenario.grid.height
            elif this_step_action == 'INDISC-REPR':
                hmap_symbol = [106] * scenario.grid.width * scenario.grid.height

        elif this_step_action in ['TARG-CONC','TARG-REPR']:
            max_num_govt_actions += 1
            x, y = govt_action_df.action_loc.item()
            hmap_x = [scen_xs[x]]
            hmap_y = [scen_ys[y]]
            hmap_size = [50]
            if this_step_action == 'TARG-CONC':
                hmap_symbol = [105]
            elif this_step_action == 'TARG-REPR':
                hmap_symbol = [106]

        max_y_govtactions.append(max_num_govt_actions)
        hMap_act_x.append(hmap_x)
        hMap_act_y.append(hmap_y)
        hMap_act_size.append(hmap_size)
        hMap_act_symbol.append(hmap_symbol)


    # ---- Create lineplot agents and attacks arrays --- #
    
    model_df = scenario.datacollector.get_model_vars_dataframe()
    
    max_y_agentattacks = list(model_df.max(axis=1))
    lPlot_nAttacks = list(model_df['num_attacks'])
    lPlot_nAgents = list(model_df['num_agents'])


    return_vals = {'step':step,
                    'lPlot_nAttacks':lPlot_nAttacks,
                    'lPlot_nAgents':lPlot_nAgents,
                    'hMap_act_x':hMap_act_x,
                    'hMap_act_y':hMap_act_y,
                    'hMap_act_size':hMap_act_size,
                    'hMap_act_symbol':hMap_act_symbol,
                    'max_y_agentattacks':max_y_agentattacks,
                    'max_y_govtactions':max_y_govtactions,
                    'hMap_sympathy':hMap_sympathy,
                    'hMap_num_combats':hMap_num_combats,
                    'pChart_classes':pChart_classes}

    return return_vals

In [50]:
sim_json = {
        'violence_prob': 0.0005,
        'gov_policy': "None",
        'reactive_level': "High",
        'discontent': "Mid",
        'starting_pop': 200,
        'total_steps': 30,
        'add_violence_aftermath':10,
        'crowding_threshold':30, 
        'agent_birth_rate':0.03
    }

In [53]:
test_resp = counterterror_model_http(sim_json)

/tmp/ipykernel_32182/3936287442.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  agents_nogov_df['step_cat'] = agents_nogov_df.Step.astype('str')
/tmp/ipykernel_32182/3936287442.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  max_sents_nodups['step'] = max_sents_nodups.step_cat.astype('int')


mod step 1 type <class 'int'>
pChart classes to add [55, 49, 42, 54]
hMap sympathy to append [[0.75 0.25 0.25]
 [0.75 0.5  0.  ]]
hMap num combatants to append [[12.  8. 10.]
 [11.  8.  5.]]
this step action TARG-REPR
mod step 2 type <class 'int'>
pChart classes to add [52, 48, 39, 12]
hMap sympathy to append [[0.25 0.   0.25]
 [0.   0.   0.  ]]
hMap num combatants to append [[3. 1. 0.]
 [5. 1. 2.]]
this step action INDISC-REPR
mod step 3 type <class 'int'>
pChart classes to add [28, 59, 30, 30]
hMap sympathy to append [[0.25 0.25 0.25]
 [0.   0.25 0.  ]]
hMap num combatants to append [[5. 5. 4.]
 [4. 7. 5.]]
this step action INDISC-REPR
mod step 4 type <class 'int'>
pChart classes to add [23, 60, 34, 25]
hMap sympathy to append [[0.25 0.25 0.25]
 [0.25 0.25 0.  ]]
hMap num combatants to append [[4. 4. 5.]
 [2. 7. 3.]]
this step action TARG-REPR
mod step 5 type <class 'int'>
pChart classes to add [25, 59, 34, 3]
hMap sympathy to append [[0.25 0.25 0.25]
 [0.25 0.25 0.  ]]
hMap num comb

In [54]:
import json
with open("test_sim_response.json","w") as f:
    json.dump(test_resp, f)

TypeError: Object of type int64 is not JSON serializable

In [2]:
scenario = model.CounterterrorismModel(N=200, height=3,width=4,prob_violence=0.0005,policy='NONE',reactive_lvl='none',discontent='mid')
for i in range(700):
    scenario.step()